In [1]:
from itertools import permutations

import pandas as pd

from glicko_rating_matches import GlickoSystem, Team

pd.set_option("display.max_columns", None)

In [2]:
def get_team_id(teams_id, team_name):
    return teams_id.loc[teams_id["team"] == team_name, "team_id"].iloc[0]

In [3]:
teams_id = pd.read_csv("./csv/teams_id.csv")
ratings = pd.read_csv(
    "./csv/glicko_rating/glicko_rating_2023.csv", parse_dates=["last_game_date"]
)
teams_id.shape, ratings.shape

((752, 2), (595, 15))

In [4]:
ratings.loc[(ratings["league"] == "LCK") & (ratings["last_game_date"].dt.year == 2023)]

,team_id,team,league,games,win,loss,winrate,streak,point,rd,sigma,error,error_square,last_game_date,change
1,69,T1,LCK,1434,951,483,0.663180,0,2013.428997,70.973133,0.558747,567.278532,299.815549,2023-05-20 13:55:00,- (-)
2,220,Gen.G,LCK,1012,604,408,0.596838,0,1998.436509,70.063440,0.561825,428.769897,227.677029,2023-05-19 14:01:00,- (-)
4,219,KT Rolster,LCK,1045,619,426,0.592344,0,1918.340377,71.018463,0.559625,434.152199,230.697547,2023-04-08 09:04:00,- (-)
7,508,Dplus KIA,LCK,617,433,184,0.701783,0,1861.164882,71.581888,0.592432,228.490182,119.344572,2023-03-23 10:44:00,- (-)
11,205,Hanwha Life Esports,LCK,853,423,430,0.495897,0,1807.642469,69.016406,0.571720,362.689244,191.382484,2023-04-02 09:15:00,- (-)
17,509,Liiv SANDBOX,LCK,445,213,232,0.478652,0,1769.155880,69.014914,0.592139,197.313719,104.193254,2023-03-22 10:36:00,- (-)
42,33,DRX,LCK,1098,571,527,0.520036,0,1689.232790,61.622956,0.494025,439.096256,225.475688,2023-03-19 09:26:00,- (-)
47,301,Kwangdong Freecs,LCK,901,426,475,0.472808,0,1677.604093,61.240961,0.484499,403.073192,215.712447,2023-03-17 09:05:00,- (-)
98,15,BRION,LCK,487,176,311,0.361396,0,1604.112090,68.337368,0.491438,200.968887,107.645277,2023-03-17 11:53:00,- (-)
112,524,Nongshim RedForce,LCK,297,114,183,0.383838,0,1588.102097,36.706755,0.486656,127.251513,68.753829,2023-03-19 06:52:00,-1 (-)


In [5]:
ratings.loc[(ratings["league"] == "LPL") & (ratings["last_game_date"].dt.year == 2023)]

,team_id,team,league,games,win,loss,winrate,streak,point,rd,sigma,error,error_square,last_game_date,change
0,444,JD Gaming,LPL,723,426,297,0.589212,0,2068.062864,70.617022,0.562817,296.388970,151.162929,2023-05-21 13:44:00,- (-)
3,458,Bilibili Gaming,LPL,537,277,260,0.515829,0,1969.908501,69.877973,0.539459,245.900041,128.682662,2023-05-21 13:44:00,- (-)
9,170,EDward Gaming,LPL,1191,736,455,0.617968,0,1833.041576,71.958942,0.571624,496.150466,261.452717,2023-04-11 13:07:00,- (-)
13,244,LNG Esports,LPL,823,412,411,0.500608,0,1789.394784,72.611305,0.591977,379.864542,202.438995,2023-04-04 12:11:00,- (-)
18,60,Oh My God,LPL,1029,474,555,0.460641,0,1765.813370,71.448747,0.559609,436.201612,223.749724,2023-04-08 13:18:00,- (-)
19,461,Top Esports,LPL,677,389,288,0.574594,0,1755.243318,72.057753,0.580066,273.738182,139.611063,2023-04-02 12:50:00,- (-)
27,445,Weibo Gaming,LPL,557,302,255,0.542190,0,1727.553177,72.560904,0.594536,252.706639,134.949453,2023-04-03 10:59:00,- (-)
30,305,Royal Never Give Up,LPL,1143,739,404,0.646544,0,1714.875803,72.461432,0.583447,473.734041,251.142597,2023-04-01 12:05:00,- (-)
80,459,FunPlus Phoenix,LPL,658,397,261,0.603343,0,1626.453182,80.395805,0.597711,271.168330,143.217598,2023-05-29 13:08:00,+40 (+50)
82,515,Ninjas in Pyjamas.CN,LPL,364,154,210,0.423077,0,1625.352528,69.439672,0.508782,159.379559,86.977694,2023-03-25 13:00:00,- (-)


In [6]:
ratings.loc[(ratings["league"] == "LEC") & (ratings["last_game_date"].dt.year == 2023)]

,team_id,team,league,games,win,loss,winrate,streak,point,rd,sigma,error,error_square,last_game_date,change
6,211,G2 Esports,LEC,1013,578,435,0.570582,0,1861.261064,71.847515,0.564581,399.605952,209.608589,2023-05-16 15:11:00,- (-)
38,333,MAD Lions,LEC,676,334,342,0.494083,0,1695.711431,65.737939,0.584468,301.907099,160.869161,2023-05-13 17:05:00,- (-)
44,598,Team BDS,LEC,72,30,42,0.416667,0,1680.847413,72.360829,0.582511,30.183166,16.469519,2023-04-23 19:40:00,- (-)
71,334,Team Vitality,LEC,386,172,214,0.445596,0,1637.948485,72.005713,0.584933,172.738544,91.498237,2023-04-22 18:48:00,+1 (-)
97,3,Fnatic,LEC,1243,758,485,0.609815,0,1605.252338,72.620249,0.587580,512.537617,271.612358,2023-04-15 17:52:00,- (-)
120,611,KOI (Spanish Team),LEC,38,17,21,0.447368,0,1577.540826,60.766348,0.579427,18.129202,10.189942,2023-04-16 19:37:00,-1 (-)
121,310,Astralis,LEC,457,225,232,0.492341,0,1574.429320,71.708896,0.584751,182.310327,92.462197,2023-04-16 17:06:00,- (-)
147,42,SK Gaming,LEC,395,184,211,0.465823,0,1546.525845,57.558267,0.591374,186.176098,99.867698,2023-04-15 19:46:00,- (-)
198,518,Excel Esports,LEC,173,64,109,0.369942,0,1514.252750,72.256232,0.578864,75.091036,40.440209,2023-03-27 15:58:00,- (-)
279,612,Team Heretics,LEC,24,8,16,0.333333,0,1478.673501,66.041777,0.575617,11.151701,5.991388,2023-03-27 21:00:00,- (-)


In [7]:
ratings.loc[(ratings["league"] == "LCS") & (ratings["last_game_date"].dt.year == 2023)]

,team_id,team,league,games,win,loss,winrate,streak,point,rd,sigma,error,error_square,last_game_date,change
14,86,Cloud9,LCS,1085,605,480,0.557604,0,1789.030355,59.919373,0.574978,451.609371,233.146775,2023-05-17 13:43:00,- (-)
29,470,Golden Guardians,LCS,287,111,176,0.386760,0,1721.213899,70.864284,0.558666,127.402489,67.959216,2023-05-14 16:36:00,- (-)
39,163,Evil Geniuses.NA,LCS,366,192,174,0.524590,0,1691.744681,67.619417,0.585426,166.852126,87.636594,2023-03-31 22:45:00,- (-)
65,414,FlyQuest,LCS,417,190,227,0.455635,0,1650.145277,70.888778,0.574269,194.618819,102.510722,2023-04-09 00:07:00,- (-)
87,264,Team Liquid,LCS,860,496,364,0.576744,0,1614.105039,66.932791,0.554640,373.434404,199.321935,2023-03-17 23:09:00,- (-)
107,468,100 Thieves,LCS,444,204,240,0.459459,0,1598.070175,69.785580,0.582777,181.145054,89.932534,2023-03-26 00:23:00,-1 (-)
128,1,Counter Logic Gaming,LCS,696,336,360,0.482759,0,1564.587089,67.193728,0.584382,321.127325,167.945502,2023-03-26 22:57:00,- (-)
148,4,TSM,LCS,1153,695,458,0.602775,0,1546.476212,69.017374,0.555351,448.445440,236.088101,2023-03-17 22:08:00,- (-)
531,350,Immortals,LCS,337,166,171,0.492582,0,1379.860678,56.593654,0.548881,147.025587,75.752154,2023-03-18 01:18:00,- (-)
546,31,Dignitas,LCS,489,206,283,0.421268,0,1369.315275,66.060757,0.548726,227.131221,119.054176,2023-03-18 00:15:00,- (-)


In [8]:
ratings.loc[
    (ratings["league"].isin(["LCK", "LPL", "LEC", "LCS"]))
    & (ratings["last_game_date"].dt.year == 2023)
]

,team_id,team,league,games,win,loss,winrate,streak,point,rd,sigma,error,error_square,last_game_date,change
0,444,JD Gaming,LPL,723,426,297,0.589212,0,2068.062864,70.617022,0.562817,296.388970,151.162929,2023-05-21 13:44:00,- (-)
1,69,T1,LCK,1434,951,483,0.663180,0,2013.428997,70.973133,0.558747,567.278532,299.815549,2023-05-20 13:55:00,- (-)
2,220,Gen.G,LCK,1012,604,408,0.596838,0,1998.436509,70.063440,0.561825,428.769897,227.677029,2023-05-19 14:01:00,- (-)
3,458,Bilibili Gaming,LPL,537,277,260,0.515829,0,1969.908501,69.877973,0.539459,245.900041,128.682662,2023-05-21 13:44:00,- (-)
4,219,KT Rolster,LCK,1045,619,426,0.592344,0,1918.340377,71.018463,0.559625,434.152199,230.697547,2023-04-08 09:04:00,- (-)
6,211,G2 Esports,LEC,1013,578,435,0.570582,0,1861.261064,71.847515,0.564581,399.605952,209.608589,2023-05-16 15:11:00,- (-)
7,508,Dplus KIA,LCK,617,433,184,0.701783,0,1861.164882,71.581888,0.592432,228.490182,119.344572,2023-03-23 10:44:00,- (-)
9,170,EDward Gaming,LPL,1191,736,455,0.617968,0,1833.041576,71.958942,0.571624,496.150466,261.452717,2023-04-11 13:07:00,- (-)
11,205,Hanwha Life Esports,LCK,853,423,430,0.495897,0,1807.642469,69.016406,0.571720,362.689244,191.382484,2023-04-02 09:15:00,- (-)
13,244,LNG Esports,LPL,823,412,411,0.500608,0,1789.394784,72.611305,0.591977,379.864542,202.438995,2023-04-04 12:11:00,- (-)


In [9]:
leagues = ["LCK", "LPL", "LEC", "LCS"]
for league in leagues:
    print(league)
    lst = ratings.loc[
        (ratings["league"] == league) & (ratings["last_game_date"].dt.year == 2023),
        "team",
    ].unique()
    print(", ".join(map(lambda x: f"'{x}'", lst)))

LCK
'T1', 'Gen.G', 'KT Rolster', 'Dplus KIA', 'Hanwha Life Esports', 'Liiv SANDBOX', 'DRX', 'Kwangdong Freecs', 'BRION', 'Nongshim RedForce'
LPL
'JD Gaming', 'Bilibili Gaming', 'EDward Gaming', 'LNG Esports', 'Oh My God', 'Top Esports', 'Weibo Gaming', 'Royal Never Give Up', 'FunPlus Phoenix', 'Ninjas in Pyjamas.CN', 'Invictus Gaming', 'Team WE', 'ThunderTalk Gaming', 'Rare Atom', 'LGD Gaming', 'Anyone's Legend', 'Ultra Prime'
LEC
'G2 Esports', 'MAD Lions', 'Team BDS', 'Team Vitality', 'Fnatic', 'KOI (Spanish Team)', 'Astralis', 'SK Gaming', 'Excel Esports', 'Team Heretics'
LCS
'Cloud9', 'Golden Guardians', 'Evil Geniuses.NA', 'FlyQuest', 'Team Liquid', '100 Thieves', 'Counter Logic Gaming', 'TSM', 'Immortals', 'Dignitas'


In [10]:
msi_teams = [
    "T1",
    "JD Gaming",
    "Bilibili Gaming",
]
ratings.loc[ratings["team"].isin(msi_teams)]

,team_id,team,league,games,win,loss,winrate,streak,point,rd,sigma,error,error_square,last_game_date,change
0,444,JD Gaming,LPL,723,426,297,0.589212,0,2068.062864,70.617022,0.562817,296.388970,151.162929,2023-05-21 13:44:00,- (-)
1,69,T1,LCK,1434,951,483,0.663180,0,2013.428997,70.973133,0.558747,567.278532,299.815549,2023-05-20 13:55:00,- (-)
3,458,Bilibili Gaming,LPL,537,277,260,0.515829,0,1969.908501,69.877973,0.539459,245.900041,128.682662,2023-05-21 13:44:00,- (-)


In [12]:
team_names = {
    # LCK
    "T1": "T1",
    "GEN": "Gen.G",
    "DK": "Dplus KIA",
    "LSB": "Liiv SANDBOX",
    "KT": "KT Rolster",
    "HLE": "Hanwha Life Esports",
    "BRO": "BRION",
    "DRX": "DRX",
    "NS": "Nongshim RedForce",
    "KDF": "Kwangdong Freecs",
    # LPL
    "BLG": "Bilibili Gaming",
    "JDG": "JD Gaming",
    # LEC
    "G2": "G2 Esports",
    "MAD": "MAD Lions",
    # LCS
    "C9": "Cloud9",
    "GG": "Golden Guardians",
}
teams = {}
lst_team = []
for name in team_names.values():
    team_id = get_team_id(teams_id, name)
    row = ratings.loc[ratings["team"] == name].iloc[0]
    teams[team_id] = Team(
        name=name,
        league=row["league"],
        team_id=row["team_id"],
        win=row["win"],
        loss=row["loss"],
        streak=row["streak"],
        point=row["point"],
        rd=row["rd"],
        sigma=row["sigma"],
        last_game_date=row["last_game_date"],
    )
    lst_team.append(teams[team_id])

In [13]:
glicko = GlickoSystem()

In [14]:
matches = [
    ("T1", "BLG"),
]

columns = ["Team1", "Point1", "Winprob1", "sum", "Winprob2", "Point2", "Team2"]
probs = pd.DataFrame(columns=columns)
for team1_name, team2_name in matches:
    team1 = teams[get_team_id(teams_id, team_names[team1_name])]
    team2 = teams[get_team_id(teams_id, team_names[team2_name])]
    lst = [
        team1.name,
        team1.point,
        team1.get_win_probability(glicko, team2) * 100,
        0,
        team2.get_win_probability(glicko, team1) * 100,
        team2.point,
        team2.name,
    ]
    df = pd.DataFrame(data=[lst], columns=columns)
    probs = pd.concat([probs, df], ignore_index=True)
probs["sum"] = probs[["Winprob1", "Winprob2"]].sum(axis=1)
probs

,Team1,Point1,Winprob1,sum,Winprob2,Point2,Team2
0,T1,2013.428997,56.084242,100.004456,43.920213,1969.908501,Bilibili Gaming


In [15]:
columns = ["Team1", "Point1", "2:0", "2:1", "1:2", "0:2", "Point2", "Team2"]
probs = pd.DataFrame(columns=columns)
for team1_name, team2_name in matches:
    team1 = teams[get_team_id(teams_id, team_names[team1_name])]
    team2 = teams[get_team_id(teams_id, team_names[team2_name])]
    team1_prob = team1.get_win_probability(glicko, team2)
    team2_prob = team2.get_win_probability(glicko, team1)
    lst = [
        team1.name,
        team1.point,
        team1_prob**2 * 100,
        (team1_prob**2) * team2_prob * 2 * 100,
        (team2_prob**2) * team1_prob * 2 * 100,
        team2_prob**2 * 100,
        team2.point,
        team2_name,
    ]
    df = pd.DataFrame(data=[lst], columns=columns)
    probs = pd.concat([probs, df], ignore_index=True)
probs["Winprob1"] = probs[["2:0", "2:1"]].sum(axis=1)
probs["Winprob2"] = probs[["1:2", "0:2"]].sum(axis=1)
columns = [
    "Team1",
    "Point1",
    "Winprob1",
    "2:0",
    "2:1",
    "1:2",
    "0:2",
    "Winprob2",
    "Point2",
    "Team2",
]
probs[columns]

,Team1,Point1,Winprob1,2:0,2:1,1:2,0:2,Winprob2,Point2,Team2
0,T1,2013.428997,59.084121,31.454422,27.629699,21.637134,19.289851,40.926985,1969.908501,BLG


In [16]:
columns = [
    "Team1",
    "Point1",
    "Winprob1",
    "3:0",
    "3:1",
    "3:2",
    "2:3",
    "1:3",
    "0:3",
    "Winprob2",
    "Point2",
    "Team2",
]
probs = pd.DataFrame(columns=columns)
for team1_name, team2_name in matches:
    team1 = teams[get_team_id(teams_id, team_names[team1_name])]
    team2 = teams[get_team_id(teams_id, team_names[team2_name])]
    team1_prob = team1.get_win_probability(glicko, team2)
    team2_prob = team2.get_win_probability(glicko, team1)
    lst = [
        team1.name,
        team1.point,
        0,
        team1_prob**3 * 100,
        (team1_prob**3) * team2_prob * 3 * 100,
        (team1_prob**3) * (team2_prob**2) * 6 * 100,
        (team2_prob**3) * (team1_prob**2) * 6 * 100,
        (team2_prob**3) * team1_prob * 3 * 100,
        team2_prob**3 * 100,
        0,
        team2.point,
        team2.name,
    ]
    df = pd.DataFrame(data=[lst], columns=columns)
    probs = pd.concat([probs, df], ignore_index=True)
probs["Winprob1"] = probs[["3:0", "3:1", "3:2"]].sum(axis=1)
probs["Winprob2"] = probs[["2:3", "1:3", "0:3"]].sum(axis=1)
probs

,Team1,Point1,Winprob1,3:0,3:1,3:2,2:3,1:3,0:3,Winprob2,Point2,Team2
0,T1,2013.428997,61.302342,17.640974,23.243861,20.417506,15.989183,14.254613,8.472144,38.71594,1969.908501,Bilibili Gaming
